In [ ]:
"""
--------------------------------------------------------------------
Author: Aaron Valoroso
Date: July 16th, 2018
License: MIT
Email: valoroso99@gmail.com
---------------------------------------------------------------------
"""
# This will only work if you assume that the Mercator projection has equal distances between each latitude gratitude.
import numpy as np
from itertools import product

url = 'http://c.tile.openstreetmap.org/{0}/{1}/{2}.png'
zoom_level = 0

long_1= -89.033203
lat_1 = 31.278551

long_2 = -72.685547
lat_2 = 23.966176

# Scrub our points and get them into our format.
the_lats = [lat_1, lat_2]
the_lats = [np.around((90 - y), decimals=2) for y in the_lats]
the_lats.sort()
print(the_lats)

the_longs = [long_1, long_2]
the_longs = [np.around((180 + x), decimals=2) for x in the_longs]
the_longs.sort()
print(the_longs)

# Firgure out our zoom points
our_y = (170/(2**zoom_level))
our_x = (360/(np.power(2, zoom_level)))
print(our_y)
print(our_x)

# Firgure out our tles
tile_lats = [int(y / our_y) for y in the_lats]
tile_longs = [int(x / our_x) for x in the_longs]
print(tile_lats)
print(tile_longs)

# Get the range of tiles
lat_range = range(tile_lats[0], (tile_lats[1] + 1))
long_range = range(tile_longs[0], (tile_longs[1] + 1))
print(lat_range)
print(long_range)

# Get the pictures
for x, y in product(long_range, lat_range):
    print(x,y)
    print(url.format(zoom_level, x, y))

In [ ]:
# What we used to get the equations below. https://en.wikipedia.org/wiki/Web_Mercator
import numpy as np
from itertools import product

url = 'http://c.tile.openstreetmap.org/{0}/{1}/{2}.png'
zoom_level = 10

long_1= -89.033203
lat_1 = 31.278551

long_2 = -72.685547
lat_2 = 23.966176

the_lats = [lat_1, lat_2]
the_lats = [int(1/(2*np.pi)*2**zoom_level*(np.pi-np.log(np.tan(np.pi/4+np.radians(y)/2)))) for y in the_lats]
print(the_lats)

# Scrub our points and get them into our format.
the_longs = [long_1, long_2]
the_longs = [int(2**(zoom_level-1)*(x/180+1)) for x in the_longs]
print(the_longs)

# Get the range of tiles
lat_range = range(the_lats[0], (the_lats[1] + 1))
long_range = range(the_longs[0], (the_longs[1] + 1))
print(lat_range)
print(long_range)

# Get the pictures
for x, y in product(long_range, lat_range):
    print(x,y)
    print(url.format(zoom_level, x, y))

In [ ]:
import requests
def download(destination, url, filename):
    temp_path = None
    if destination[-1] != '/':
        destination += '/'

    if os.path.exists(destination):
        response = requests.get(url, verify=False)
        temp_path = destination + filename
        if response.status_code == 200:
            # Have to download the files in order, not when they finish. Download each file at a time.
            open(temp_path, 'wb').write(response.content)
#             chunk_size = 64 * 1024
#             with open(temp_path, 'wb') as f:
#                 for content in response.iter_content(chunk_size):
#                     f.write(content)
    else:
        raise ValueError("Sorry, but the download destination that you requested is not a path.")

In [ ]:
from PIL import Image
def image_stitch(dir_path):
    
    dir_path += '/'
    
    #----------------------------------------------------------------------------
    list_of_pictures = list()
    lists_of_list_of_pictures = list()
    list_iterator = 0
    stored_number = ''
    temp_number = ''
    
    for iterator, filename in enumerate(os.listdir(dir_path)):
        temp_number = filename[0] + filename[1]
        if iterator == 0:
            stored_number = temp_number
            
        if temp_number != stored_number:
            lists_of_list_of_pictures.append(list_of_pictures)
            stored_number = temp_number
            list_of_pictures = list()
            
        list_of_pictures.append(filename)

    lists_of_list_of_pictures.append(list_of_pictures)
    stored_number = temp_number
    #----------------------------------------------------------------------------
    total_width_of_list = 0
    tile_width = 0
    growing_width = 0
    
    total_height_of_list = 0
    tile_height = 0
    growing_height = 0

    heights = list()
    widths = list()
    stitched_image = None
    new_image = None
    creating_a_bigger_picture = None
    
    for item in lists_of_list_of_pictures:
        images = [ Image.open(dir_path + i) for i in item ]

        for second_item in images:
            (width, height) = second_item.size
            widths.append(width)
            heights.append(height)
        
        # We need to make sure that the individual height and width of each time is an int and not a float.
        # Hence why I had to do the int() around our division. 
        total_height_of_list = sum(heights)
        tile_height = int(total_height_of_list/len(heights))
        total_width_of_list = sum(widths)
        tile_width = int(total_width_of_list/len(widths))

        # Create a new image for our picture. So our width is not going to change but our height 
        # will. Our height is the number of tiles on top of one another.        
        new_image = Image.new('RGB', (tile_width, total_height_of_list))

        # Here I am putting the rows of tiles together. 
        for iterator2, item in enumerate(images):
            if iterator2 == 0:
                new_image.paste(im=item, box=(0,0))
            else:
                growing_height += 256
                new_image.paste(im=item, box=(0, growing_height))
            
#       Put our two stacked images together into one image.
        if stitched_image is None:
            stitched_image = new_image
        else:
            growing_width += 256
            creating_a_bigger_picture = Image.new('RGB', ((total_width_of_list+256), total_height_of_list))
            (width, height) = new_image.size
            creating_a_bigger_picture.paste(im=stitched_image, box=(0,0))
            creating_a_bigger_picture.paste(im=new_image, box=(growing_width,0))
            stitched_image = creating_a_bigger_picture

        widths = list()
        heights = list()
        images = list()
        new_image = None
        creating_bigger_picture = None
        growing_height = 0 

    stitched_image.show()
    
    return

In [ ]:
# What we used to get the equations below. https://en.wikipedia.org/wiki/Web_Mercator
import numpy as np
import shutil
import os
from itertools import product

switch = False

if switch == True:
    url = 'http://c.tile.openstreetmap.org/{0}/{1}/{2}.png'
    zoom_level = 7

    long_1= -89.033203
    lat_1 = 31.278551

    long_2 = -72.685547
    lat_2 = 23.966176

    the_lats = [lat_1, lat_2]
    the_lats = [int(1/(2*np.pi)*2**zoom_level*(np.pi-np.log(np.tan(np.pi/4+np.radians(y)/2)))) for y in the_lats]

    the_longs = [long_1, long_2]
    the_longs = [int(2**(zoom_level-1)*(x/180+1)) for x in the_longs]

    # Get the range of tiles
    lat_range = range(the_lats[0], (the_lats[1] + 1))
    long_range = range(the_longs[0], (the_longs[1] + 1))

dir_path = os.getcwd()
dir_path += '/image_example'

if switch == True:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    else:
        shutil.rmtree(dir_path)
        os.makedirs(dir_path)

    # Get the pictures
    for x, y in product(long_range, lat_range):
        filename = "{0}-{1}.png".format(x,y)
        download(dir_path, url.format(zoom_level, x, y), filename)

image_stitch(dir_path)

In [ ]:
# Version-1
#                 stored_number = int(images[0][0])
#                 image1 = Image.open(dir_path + images[0])
#                 image2 = Image.open(dir_path + images[1])
#                 (width1, height1) = image1.size
#                 (width2, height2) = image2.size
#                 result_width = width1 + width2
#                 result_height = height1 + height2
                
#                 result = Image.new('RGB', (width1, result_height))
#                 result.paste(im=image1, box=(0,0,width1,height1))
#                 result.paste(im=image2, box=(0,height1,width1, result_height))
#                 result.show()

# Version-2
#     for iterator, filename in enumerate(os.listdir(dir_path)):
#         images.append(filename)
#         if iterator % 2 == 0:
#             if iterator == 2:
#                 imgs = [ Image.open(dir_path + i) for i in images ]
#                 for item in imgs:
#                     (width, height) = item.size
#                     widths.append(width)
#                     heights.append(height)
                
#                 total_height = sum(heights)
#                 total_width = sum(widths)
#                 print(total_height)
#                 result = Image.new('RGB', (int(total_width/len(widths)), total_height))
                
#                 for iterator, item in enumerate(imgs):
#                     the_height += 256
#                     if iterator == 0:
#                         result.paste(im=item, box=(0,0,256,256))
#                     else:
#                         result.paste(im=item, box=(0,(the_height - 256),256,the_height))
                
#                 result.show()
                
#                 return

#Version-3
#     images = list()
#     heights = list()
#     widths = list()
    
#     stitched_image = None
#     stored_number = ''
#     temp_number = ''
#     dir_path += '/'
#     bigger_picture_width = 0
#     bigger_picture_height = 0
    
#     total_width = 0
#     total_height = 0
#     the_height = 0

#     for iterator, filename in enumerate(os.listdir(dir_path)):
#         temp_number = filename[0] + filename[1]
#         if iterator == 0:
#             stored_number = temp_number
            
#         if temp_number != stored_number:
#             imgs = [ Image.open(dir_path + i) for i in images ]
#             for item in imgs:
#                 (width, height) = item.size
#                 widths.append(width)
#                 heights.append(height)

#             total_height = sum(heights)
#             total_width = sum(widths)

#             result = Image.new('RGB', (int(total_width/len(widths)), total_height))

#             for iterator, item in enumerate(imgs):
#                 the_height += 256
#                 if iterator == 0:
#                     result.paste(im=item, box=(0,0,256,256))
#                 else:
#                     result.paste(im=item, box=(0,(the_height - 256),256,the_height))

#             # Put our two stacked images together
#             if stitched_image is None:
#                 stitched_image = result
#             else:
#                 bigger_picture_width += 256
#                 new_result = Image.new('RGB', ((total_width+256), total_height))
#                 (width, height) = result.size
#                 new_result.paste(im=stitched_image, box=(0,0))
#                 new_result.paste(im=result, box=(bigger_picture_width,0))
#                 stitched_image = new_result

#             stored_number = temp_number
#             widths[:] = []
#             heights[:] = []
#             imgs[:] = []
#             images[:] = []
#             result = None
#             the_height = 0 
        
#         images.append(filename)

#     stitched_image.show()

#Version-4
    
#     dir_path += '/'
    
#     #----------------------------------------------------------------------------
#     whole_directory = []
#     for filename in os.listdir(dir_path):
#         filename2, file_extension = os.path.splitext(filename)
#         filename2 += '1'
#         whole_directory.append(float(filename2.replace('-', '.')))

#     whole_directory.sort()
    
#     new_directory = list()
#     for filename in whole_directory:
#         filename = str(filename)
#         filename = filename.replace('.', '-')
#         filename = filename[:-1]
#         filename += '.png'
#         new_directory.append(filename)
#     #----------------------------------------------------------------------------
#     if len(new_directory) > 1:
#         list_of_pictures = list()
#         lists_of_list_of_pictures = list()
#         list_iterator = 0
#         stored_number = ''
#         temp_number = ''

#         for iterator, filename in enumerate(new_directory):
#             first_part, second_part = filename.split('-')
#             temp_number = first_part
#             if iterator == 0:
#                 stored_number = temp_number

#             if temp_number != stored_number:
#                 lists_of_list_of_pictures.append(list_of_pictures)
#                 stored_number = temp_number
#                 list_of_pictures = list()

#             list_of_pictures.append(filename)

#         lists_of_list_of_pictures.append(list_of_pictures)
#         stored_number = temp_number
#         #----------------------------------------------------------------------------
#         total_width_of_list = 0
#         tile_width = 0
#         growing_width = 0

#         total_height_of_list = 0
#         tile_height = 0
#         growing_height = 0

#         heights = list()
#         widths = list()
#         stitched_image = None
#         new_image = None
#         creating_a_bigger_picture = None

#         for item in lists_of_list_of_pictures:
#             images = [ Image.open(dir_path + i) for i in item ]

#             for second_item in images:
#                 (width, height) = second_item.size
#                 widths.append(width)
#                 heights.append(height)

#             # We need to make sure that the individual height and width of each time is an int and not a float.
#             # Hence why I had to do the int() around our division. 
#             total_height_of_list = sum(heights)
#             tile_height = int(total_height_of_list/len(heights))
#             total_width_of_list = sum(widths)
#             tile_width = int(total_width_of_list/len(widths))

#             # Create a new image for our picture. So our width is not going to change but our height 
#             # will. Our height is the number of tiles on top of one another.        
#             new_image = Image.new('RGB', (tile_width, total_height_of_list))

#             # Here I am putting the rows of tiles together. 
#             for iterator2, item in enumerate(images):
#                 if iterator2 == 0:
#                     new_image.paste(im=item, box=(0,0))
#                 else:
#                     growing_height += 256
#                     new_image.paste(im=item, box=(0, growing_height))

#             # Put our two stacked row images together into one image.
#             if stitched_image is None:
#                 stitched_image = new_image
#             else:
#                 growing_width += 256
#                 creating_a_bigger_picture = Image.new('RGB', ((total_width_of_list+256), total_height_of_list))
#                 (width, height) = new_image.size
#                 creating_a_bigger_picture.paste(im=stitched_image, box=(0,0))
#                 creating_a_bigger_picture.paste(im=new_image, box=(growing_width,0))
#                 stitched_image = creating_a_bigger_picture

#             widths = list()
#             heights = list()
#             images = list()
#             new_image = None
#             creating_bigger_picture = None
#             growing_height = 0 
#         stitched_image.show()
#     #----------------------------------------------------------------------------

#     return

#Version-5
# def image_stitch(current_path, max_width, max_height, min_width, min_height, images):
#     # Create the new image with the incoming max width and height.
#     stitched_image = Image.new('RGB', (max_width, max_height))
    
#     for file in images:
#         incoming_image = Image.open(current_path + file)
#         filename, file_extension = os.path.splitext(file)
#         x, y = filename.split('-')
#         x = (int(x) - min_width) * 256
#         y = (int(y) - min_height) * 256
#         stitched_image.paste(im=incoming_image, box=(x, y))
#         incoming_image.close()
#     stitched_image.show()

#     return